# COGS209 Project 2 


In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import Ridge,Lasso

from sklearn import linear_model
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model

from statsmodels.regression.linear_model import OLS
data=pd.read_csv('https://www.dropbox.com/scl/fi/eut3hbc6vh94713teclkd/heart.csv?rlkey=24b949anqmwjs4g9zstvdyrv0&st=xb2s3u6m&dl=0')

ModuleNotFoundError: No module named 'pandas'